In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

pdf_path = "C:/wanted/Lang/Presentation-Agent/data/pdf/DeePrint.pdf"

loader = PyMuPDFLoader(pdf_path, extract_images=True)

docs = loader.load()

# 문서 정보 출력
for idx, doc in enumerate(docs):
    print(f"Page {idx + 1}:")
    print("Text:", doc.page_content)  # 페이지의 텍스트 출력
    if "images" in doc.metadata:
        print("Images:", doc.metadata["images"])  # 이미지 정보 출력
    print("-" * 50)


In [ ]:
from dotenv import load_dotenv
load_dotenv('../../.env')

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# PDF 파일 로드 및 페이지 분할
loader = PyPDFLoader("C:/wanted/Lang/Presentation-Agent/data/pdf/DeePrint.pdf")  # PDF 파일 경로 설정
pages = loader.load_and_split()  # 페이지별로 텍스트 추출

# **2. 벡터 스토어 생성**
vectorstore = Chroma.from_documents(pages, embedding=OpenAIEmbeddings())  # 임베딩 생성 및 저장
retriever = vectorstore.as_retriever()  # 검색기 생성

# **3. 검색된 문서 데이터를 문자열로 변환하는 함수**
def format_context(docs):
    """검색된 문서 리스트를 하나의 문자열로 병합"""
    return "\n".join([doc.page_content for doc in docs])

# **4. 프롬프트 템플릿 정의**
template = '''다음 컨텍스트를 기반으로 발표 대본을 작성하세요:
{context}

질문: {question}
'''
prompt = ChatPromptTemplate.from_template(template)

# **5. LLM 모델 초기화**
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# **6. RAG 체인 구성**
rag_chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}  # 컨텍스트와 질문 연결
    | prompt  # 프롬프트 적용
    | model   # 언어 모델 실행
    | StrOutputParser()  # 결과 파싱
)

# **7. 체인 실행**
retrieved_docs = retriever.get_relevant_documents("이 문서를 기반으로 발표 대본을 작성해주세요.")  # 문서 검색
formatted_context = format_context(retrieved_docs)  # 검색된 문서를 문자열로 변환

result = rag_chain.invoke({
    "context": formatted_context,  # 문자열 형태의 context 전달
    "question": "페이지별로 발표 대본을 작성해주세요."
})

# 결과 출력
print(result)


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from PyPDF2 import PdfReader

# **1. PDF 텍스트 추출**
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"  # 각 페이지의 텍스트를 추가
    return text.strip()

# PDF 파일 경로 설정
pdf_path = "C:/wanted/Lang/Presentation-Agent/data/pdf/DeePrint.pdf"
pdf_text = extract_text_from_pdf(pdf_path)

# **2. 프롬프트 템플릿 정의**
template = '''다음 PDF 내용을 기반으로 발표 대본을 작성하세요:
{context}

질문: {question}
'''
prompt = ChatPromptTemplate.from_template(template)

# **3. LLM 모델 초기화**
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# **4. 프롬프트 실행**
question = "페이지별로 발표 대본을 작성해주세요."
formatted_prompt = prompt.format(context=pdf_text, question=question)

result = model(formatted_prompt)

# 결과 출력
print(result)


In [ ]:
print(result.content)

In [ ]:
import yt_dlp
from pydub import AudioSegment
import os

def time_convert(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

def download_audio(url, save_dir, clip_idx, start=None, end=None):
    os.makedirs(save_dir, exist_ok=True)

    # 확장자 포함된 yt-dlp 저장용 임시 파일 경로
    temp_template = os.path.join(save_dir, f"cheo_temp_{clip_idx}.%(ext)s")
    temp_wav = os.path.join(save_dir, f"cheo_temp_{clip_idx}.wav")

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': temp_template,
        'quiet': True,
        'force_ipv4': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    # 자르고 저장
    audio = AudioSegment.from_file(temp_wav, format="wav")
    if start and end:
        start_ms = time_convert(start) * 1000
        end_ms = time_convert(end) * 1000
        audio = audio[start_ms:end_ms]

    # 최종 파일명: cheo_1.wav, cheo_2.wav ...
    final_path = os.path.join(save_dir, f"winter_{clip_idx}.wav")
    audio.export(final_path, format="wav")

    os.remove(temp_wav)
    return audio


In [ ]:
url = "https://www.youtube.com/watch?v=4vS0Bhe5zjA"
save_dir = r"C:\wanted\Lang\Presentation-Agent\data\train_wav"
audio_data = []
for i in range(5):
    start = f"{12+i}:00"
    end = f"{12+i+1}:00"
    saved = download_audio(url, save_dir, clip_idx=i+1, start=start, end=end)
    audio_data.append(saved)


In [ ]:
from zonos.model import Zonos

print("Zonos import 성공")


In [ ]:
import os

os.environ["PATH"] += os.pathsep + r"C:\Program Files\eSpeak NG"
os.environ["PHONEMIZER_ESPEAK_LIBRARY"] = r"C:\Program Files\eSpeak NG\espeak.dll"

from phonemizer.backend import EspeakBackend

backend = EspeakBackend(language='ko')
print(backend.phonemize(["야 이제 되냐 안 되냐"]))

In [ ]:
import os
import torchaudio
from zonos.model import Zonos
from zonos.conditioning import make_cond_dict
import torch._dynamo  # suppress warning if needed

# ❗ phonemizer용 환경변수 설정 (espeak.dll 대응)
os.environ["PATH"] += os.pathsep + r"C:\Program Files\eSpeak NG"
os.environ["PHONEMIZER_ESPEAK_LIBRARY"] = r"C:\Program Files\eSpeak NG\espeak.dll"

# ❗ torch compile 비활성화 (C++ 컴파일러 없이 실행)
torch._dynamo.config.suppress_errors = True

# 경로 설정
path = r"C:\wanted\Lang\Presentation-Agent\data\train_wav"

# 모델 불러오기
device = "cuda"  # 또는 "cpu"
model = Zonos.from_pretrained("Zyphra/Zonos-v0.1-transformer", device=device)

# 입력 음성 로드
wav, sampling_rate = torchaudio.load(os.path.join(path, "cheo_1.wav"))

# 스피커 임베딩 생성 (제로샷 보이스 클로닝)
speaker = model.make_speaker_embedding(wav, sampling_rate)
text = """
현대 사회에서 *AI* 기술은 빠르게 발전하고 있습니다.
우리의 '발표' 능력을 지원하는 시스템이 필요합니다.
"""
# 텍스트와 스피커로 조건 딕셔너리 구성
cond_dict = make_cond_dict(text=text, speaker=speaker, language="ko")
conditioning = model.prepare_conditioning(cond_dict)

# 음성 생성 (컴파일러 비활성화)
codes = model.generate(conditioning, disable_torch_compile=True)
wavs = model.autoencoder.decode(codes).cpu()

# 결과 저장
torchaudio.save(os.path.join(path, "sample.wav"), wavs[0], model.autoencoder.sampling_rate)

print("🔊 sample.wav 생성 완료!")


In [ ]:
import os
import torchaudio
from zonos.model import Zonos
from zonos.conditioning import make_cond_dict
import torch._dynamo  # suppress warning if needed

def zero_shot(path,file,text,name):
    torch.cuda.empty_cache()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = Zonos.from_pretrained("Zyphra/Zonos-v0.1-transformer", device=device)
    wav, sampling_rate = torchaudio.load(os.path.join(path, file))
    speaker = model.make_speaker_embedding(wav, sampling_rate)

    cond_dict = make_cond_dict(
        text = text,
        speaker = speaker,
        language = "ko"
    )
    conditioning = model.prepare_conditioning(cond_dict)
    codes = model.generate(conditioning, disable_torch_compile=True)
    wavs = model.autoencoder.decode(codes).cpu()

    # 결과 저장
    torchaudio.save(os.path.join(path, f"{name}_zero_shot.wav"), wavs[0], model.autoencoder.sampling_rate)

    print("🔊 zero_shot.wav 생성 완료!")
    return codes

def few_shot(path, data, text, name):
    '''few-shot 보이스 클리닝'''
    torch.cuda.empty_cache()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = Zonos.from_pretrained("Zyphra/Zonos-v0.1-transformer", device=device)

    embeddings = []
    for file in data:
        wav, sampling_rate = torchaudio.load(os.path.join(path, file))
        emb = model.make_speaker_embedding(wav, sampling_rate)
        embeddings.append(emb)

    speaker_embedding = torch.stack(embeddings).mean(dim=0)

    cond_dict = make_cond_dict(
        text = text,
        speaker = speaker_embedding,
        language="ko"
    )
    conditioning = model.prepare_conditioning(cond_dict)
    # 음성 생성 (컴파일러 비활성화)
    codes = model.generate(conditioning, disable_torch_compile=True)
    wavs = model.autoencoder.decode(codes).cpu()

    # 결과 저장
    torchaudio.save(os.path.join(path, f"{name}_few_shot.wav"), wavs[0], model.autoencoder.sampling_rate)

    print("🔊 few_shot.wav 생성 완료!")
    return codes


In [ ]:
path = r"C:\wanted\Lang\Presentation-Agent\data\train_wav"
file = "winter_2.wav"
data = [f"winter_{i}.wav" for i in range(2,6)]
name = "winter"
text = """
현대 사회에서 *AI* 기술은 빠르게 발전하고 있습니다.
우리의 '발표' 능력을 지원하는 시스템이 필요합니다.
"""

zero_shot(path,file, text,name)
few_shot(path,data, text,name)

In [ ]:
from transformers import pipeline
import scipy
synthesiser = pipeline("text-to-speech", "suno/bark")

speech = synthesiser("현대 사회에서 발표는 필수적인 활동입니다.", forward_params={"do_sample": True})

scipy.io.wavfile.write("bark_out.wav", rate=speech["sampling_rate"], data=speech["audio"])


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from IPython.display import Audio
torch.cuda.empty_cache()

# download and load all models
preload_models(
    text_use_gpu=False,
    text_use_small=True,
    coarse_use_gpu=True,
    fine_use_gpu=True,
    codec_use_gpu=True
)


# generate audio from text
text_prompt = """현대 사회에서 발표는 필수적인 활동입니다.
하지만 많은 사람들은 내성적인 성격, 무대 공포증, 실수에 대한 두려움 등으로 인해 어려움을 겪습니다.
이러한 문제를 해결하기 위해, AI 기반 발표 지원 시스템인 "저희 발표 안합니다!" 프로젝트가 기획되었습니다."""
speech_array = generate_audio(text_prompt)

# play text in notebook
Audio(speech_array, rate=SAMPLE_RATE)


In [ ]:
# 📌 경로 설정 (MeloTTS 코드 추가)
import sys, os
sys.path.append("C:/wanted/Lang/MeloTTS")  # 너의 경로에 맞게 조정

# 📌 PyTorch 확인
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")


In [ ]:
import os

print(os.path.exists("C:/wanted/Lang/MeloTTS/melo/api.py"))
print(os.listdir("C:/wanted/Lang/MeloTTS"))


In [ ]:
import sys
sys.path.append("C:/wanted/Lang/MeloTTS/melo")  # ← melo 폴더 자체를 직접 추가

from api import TTS


In [ ]:
# 📌 Jupyter에서 재생 (IPython 오디오 위젯)
from IPython.display import Audio
Audio(output_path)

In [ ]:
from melo.api import TTS

# Speed is adjustable
speed = 1.0
device = 'cpu' # or cuda:0

text = "안녕하세요! 오늘은 날씨가 정말 좋네요."
model = TTS(language='KR', device=device)
speaker_ids = model.hps.data.spk2id

output_path = 'kr.wav'
model.tts_to_file(text, speaker_ids['KR'], output_path, speed=speed)

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
# 여기에요 여기 구글은 여기입니다

In [17]:
def synthesize_text():
    """Synthesizes speech from the input string of text."""
    from google.cloud import texttospeech

    text = "과연 성공인것인가 실패인 것인가 나는 누구인가 우린 어디로 가는 것인가 성공은 가능한 것인가가."
    client = texttospeech.TextToSpeechClient()

    input_text = texttospeech.SynthesisInput(text=text)

    # Note: the voice can also be specified by name.
    # Names of voices can be retrieved with client.list_voices().
    voice = texttospeech.VoiceSelectionParams(
        language_code="ko-KR"
    )

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    response = client.synthesize_speech(
        input=input_text,
        voice=voice,
        audio_config=audio_config,
    )

    # The response's audio_content is binary.
    with open("output.mp3", "wb") as out:
        out.write(response.audio_content)
        print('Audio content written to file "output.mp3"')


In [18]:
synthesize_text()

Audio content written to file "output.mp3"
